<a href="https://colab.research.google.com/github/Tizzy1999/ml2019/blob/master/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np

##**Dropping columns in a dataframe**

In [0]:

df = pd.read_csv('/content/gdrive/My Drive/201911ML/BL-Flickr-Images-Book.csv')

In [0]:
df.head()

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [0]:
to_drop = ['Edition Statement',
           'Corporate Author',
            'Corporate Contributors',
            'Former owner',
            'Engraver',
            'Contributors',
            'Issuance type',
            'Shelfmarks']
          
df.drop(to_drop, inplace=True, axis=1)
# specify the labels to be removed and the axis where Pandas should look for the labels
# Alternative way of dropping: 
# directly passing the columns to be removed 
# df.drop(columns=to_drop, inplace=True)


In [0]:
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


##**Changing the index of a dataframe**

In [0]:
df['Identifier'].is_unique

True

In [0]:
# replace the existing index with the 'identifier' column using set_index
# the method returns a modified copy of the object by default, 
# without making changes directly to the object. 
# instead, we can setting the inplace parameter
# df.set_index('Indentifier', inplace=True)
df = df.set_index('Identifier')
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


In [0]:
# access each record using loc, which allows label-based indexing
# it's the labeling of a record without regard to its position
df.loc[206]

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

> In other words, 206 is the ***first label*** of th index. To access it by position, we could use df.iloc[0], which does ***position-based indexing***.

## **Tidying up Fields in the Data**
###object dtype
* which is roughly analogous to ***str*** in native Python.
* It encapsulates any field that can’t be neatly fit as numerical or categorical data.



In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8287 entries, 206 to 4160339
Data columns (total 6 columns):
Place of Publication    8287 non-null object
Date of Publication     8106 non-null object
Publisher               4092 non-null object
Title                   8287 non-null object
Author                  6509 non-null object
Flickr URL              8287 non-null object
dtypes: object(6)
memory usage: 773.2+ KB


In [0]:
df.get_dtype_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: `get_dtype_counts` has been deprecated and will be removed in a future version. For DataFrames use `.dtypes.value_counts()
  """Entry point for launching an IPython kernel.


object    6
dtype: int64

In [0]:
df.loc[1905:,'Date of Publication'].head(10)

Identifier
1905           1888
1929    1839, 38-54
2836           1897
2854           1865
2956        1860-63
2957           1873
3017           1866
3131           1899
4598           1814
4884           1820
Name: Date of Publication, dtype: object

* Remove the extra dates in square brackets, wherever present: 1879 [1878]
* Convert date ranges to their “start date”, wherever present: 1860-63; 1839, 38-54
* Completely remove the dates we are not certain about and replace them with NumPy’s NaN: [1897?]
* Convert the string nan to NumPy’s NaN value

In [0]:
# synthesizing year pattern using a single regular expression
regex = r'^(\d{4})'
# \d represents ant digit, {4} denotes repeat this rule four times
# ^ character matches the start of a string
# () denotes a capturin group

In [0]:
extr = df['Date of Publication'].str.extract(regex, expand=False)
extr.head(20)

Identifier
206     1879
216     1868
218     1869
472     1851
480     1857
481     1875
519     1872
667      NaN
874     1676
1143    1679
1280    1802
1808    1859
1905    1888
1929    1839
2836    1897
2854    1865
2956    1860
2957    1873
3017    1866
3131    1899
Name: Date of Publication, dtype: object

In [0]:
# technically, the column still has object dtype
# but we can easily get its numerical version with pd.to_numeric

df['Date of Publication'] = pd.to_numeric(extr)
df['Date of Publication'].dtype

dtype('float64')

In [0]:
df['Date of Publication'].isnull().sum()/len(df)

0.11717147339205986

##**Combining str Methods with Numpy to Clean Columns**

* the ***str*** attribute of column is a way to access string operations in Pandas, such as .split(), .replace(), and .captitalize()

np.where(condition, then , else)
* condition is either an array-like object or a boolean mask. then is the value to be used if condition is true. else is the value to be used otherwise.

In [0]:
df['Place of Publication'].head(15)

Identifier
206                                  London
216                London; Virtue & Yorston
218                                  London
472                                  London
480                                  London
481                                  London
519                                  London
667     pp. 40. G. Bryan & Co: Oxford, 1898
874                                 London]
1143                                 London
1280                               Coventry
1808                            Christiania
1905                                Firenze
1929                              Amsterdam
2836                                 Savona
Name: Place of Publication, dtype: object

In [0]:
# Let's look at two specific entries:
print(df.loc[4157862])
print(df.loc[4159587])

Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                  1867
Publisher                                                      T. Fordyce
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object
Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1834
Publisher                                                Mackenzie & Dent
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4159587, dtype: object


* These two books were publiched in the same place, but with different format

In [0]:
pub = df['Place of Publication']
# get a boolean mask of the london
london = pub.str.contains('London')
print(london[:10])
# get a boolean mask of the oxford
oxford = pub.str.contains('Oxford')
print(oxford[:10])

Identifier
206      True
216      True
218      True
472      True
480      True
481      True
519      True
667     False
874      True
1143     True
Name: Place of Publication, dtype: bool
Identifier
206     False
216     False
218     False
472     False
480     False
481     False
519     False
667      True
874     False
1143    False
Name: Place of Publication, dtype: bool


In [0]:
# combine london and oxford
df['Place of Publication'] = np.where(london, 'London', 
                                      np.where(oxford, 'Oxford', 
                                               pub.str.replace('-', ' ')))
df['Place of Publication'].head(15)

Identifier
206          London
216          London
218          London
472          London
480          London
481          London
519          London
667          Oxford
874          London
1143         London
1280       Coventry
1808    Christiania
1905        Firenze
1929      Amsterdam
2836         Savona
Name: Place of Publication, dtype: object

* here, ***np.where*** function is called in a nested structure, with condition being a Series of booleans obtained with str.contains().

* ***str.contains()*** is used to find the occurence of an entity in an iterable (or substring in a string)

* we replace hyphens with a space with ***str.replace()*** and reassign to the column in our DataFrame




##**Cleaning the Entire Datase Using the applymap Function**

* ***.applymap()*** method: apply a function to all the elements in a Dataframe

After examing the original dataset:
* We see that we have periodic state names followed by the **university towns** in that state
* The state names have the **"[edit]"**substring in them
* Create a list of **(state, city) tuples** and wrapping that list in a dataframe

In [0]:
university_towns = []
with open('/content/gdrive/My Drive/201911ML/university_towns.txt') as file:
  for line in file:
    if '[edit]' in line:
      # Remember this 'state' until the next is found
      state = line
    else:
      # Otherwise, we have a city; keep 'state' as last-seen
      university_towns.append((state, line))


In [0]:
university_towns[:20]

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'),
 ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'),
 ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'),
 ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'),
 ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n'),
 ('Alabama[edit]\n', 'Troy (Troy University)[2]\n'),
 ('Alabama[edit]\n',
  'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]\n'),
 ('Alabama[edit]\n', 'Tuskegee (Tuskegee University)[5]\n'),
 ('Alaska[edit]\n', 'Fairbanks (University of Alaska Fairbanks)[2]\n'),
 ('Arizona[edit]\n', 'Flagstaff (Northern Arizona University)[6]\n'),
 ('Arizona[edit]\n', 'Tempe (Arizona State University)\n'),
 ('Arizona[edit]\n', 'Tucson (University of Arizona)\n'),
 ('Arkansas[edit]\n',
  'Arkadelphia (Henderson State University, Ouachita Baptist University)[2]\n'),
 ('Arkansas[edit]\n',
  'Conway (Central Baptist College, Hendrix College, Universit

In [0]:
# convert to pandas dataframe
towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])
towns_df.head()

,State,RegionName
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n


* We only need the state name and town name

In [0]:
# We only need state and town name, so we can remove everthing else
# In the following function, checks are performed in each element to 
# determine whether there's a ( or [ in the element or not
def get_citystate(item):
  if '(' in item:
    return item[:item.find('(')]
  elif '[' in item:
    return item[:item.find('[')]
  else:
    return item

In [0]:
# .applymap() only takes one parameter, which is the function that should
# be applied to each element
towns_df = towns_df.applymap(get_citystate)

In [0]:
towns_df.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


##**Renaming Columns and Skipping Rows**


In [0]:
olympics_df = pd.read_csv('/content/gdrive/My Drive/201911ML/olympics.csv')
olympics_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,? Summer,01 !,02 !,03 !,Total,? Winter,01 !,02 !,03 !,Total,? Games,01 !,02 !,03 !,Combined total
1,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
2,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
3,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
4,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12


if we go to the [source of this dataset](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), we'd see that 
* **NaN** above should really be something like “Country”
* **? Summer** is supposed to represent “Number of Summer Games”
* **01!** should be “Gold”, and so on.

Therefore, we need to do two things:
* skip one row and set the header as the first(0-indexed) row
* rename the columns

We can skip rows and set the header while reading the CSV file by passing some parameters to the read_csv() function.

In [0]:
olympics_df = pd.read_csv('/content/gdrive/My Drive/201911ML/olympics.csv', header=1)
olympics_df.head()

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


* We now have the correct row set as the header and all unnecessary rows removed. 
* Now, we need to rename the columns using DataFrame's rename() method, which allows you to relabel an axis based on a ***mapping***

In [0]:
new_names =  {'Unnamed: 0': 'Country',
               '? Summer': 'Summer Olympics',
               '01 !': 'Gold',
               '02 !': 'Silver',
               '03 !': 'Bronze',
               '? Winter': 'Winter Olympics',
               '01 !.1': 'Gold.1',
               '02 !.1': 'Silver.1',
               '03 !.1': 'Bronze.1',
               '? Games': '# Games',
               '01 !.2': 'Gold.2',
               '02 !.2': 'Silver.2',
               '03 !.2': 'Bronze.2'}

In [0]:
olympics_df.rename(columns=new_names, inplace=True)
olympics_df.head()

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
